## Installs and imports

In [2]:
!pip install pyscf qiskit-nature qiskit-algorithms

  Using cached pyscf-2.9.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached qiskit-2.0.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.4.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pbr-6.1.1-py2.py3-none-any.

In [3]:
from pyscf import gto, scf, ao2mo
import numpy as np
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from qiskit_aer import Aer
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.primitives import Estimator
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms.optimizers import SPSA
from IPython.display import Image

In [11]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Finding ground state of a system

In [5]:
def expand_to_spin_orbital_basis(h1_ao, eri_ao):
    n = h1_ao.shape[0]
    h1_spin = np.kron(h1_ao, np.eye(2))
    eri_spin = np.zeros((2*n, 2*n, 2*n, 2*n))
    for p in range(2*n):
        for q in range(2*n):
            for r in range(2*n):
                for s in range(2*n):
                    eri_spin[p, q, r, s] = eri_ao[p//2, q//2, r//2, s//2] * (
                        (p % 2 == r % 2) and (q % 2 == s % 2)
                    )
    return h1_spin, eri_spin

In [6]:
def get_fermionic_op(atom_string: str, spin: int, charge: int, basis: str = "sto-3g"):
    mol = gto.M(
        atom=atom_string,
        basis=basis,
        spin=spin,
        charge=charge,
        verbose=0
    )

    # Restricted hartree-fock
    mf = scf.RHF(mol)
    mf.kernel()

    # One-electron integrals (in AO basis)
    h1_ao = mf.get_hcore()

    # Two-electron integrals (in AO basis, 4-index)
    eri_ao = mol.intor('int2e', aosym='s1')  # (μν|λσ)

    # Convert to format for Qiskit
    h1_spin, eri_spin = expand_to_spin_orbital_basis(h1_ao, eri_ao)

    # Construct electronic Hamiltonian from raw integrals
    electronic_energy = ElectronicEnergy.from_raw_integrals(h1_spin, eri_spin, auto_index_order=False)
    
    # Convert to FermionicOp
    atom_hamiltonian = electronic_energy.second_q_op()

    return atom_hamiltonian

In [9]:
def simulate_fermionic_op(atom_hamiltonian: FermionicOp):
    # Convert to qubit Hamiltonian using Jordan-Wigner transformation
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(atom_hamiltonian)
    
    # Define the quantum ansatz
    ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=2)
    
    backend = Aer.get_backend('statevector_simulator')
    estimator = Estimator()
    optimizer = SPSA()  # Create an instance of the optimizer
    vqe = VQE(estimator, ansatz, optimizer)  # Pass the optimizer to VQE
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    return result.eigenvalue.real

In [12]:
StringLi2S = """
    Li 0.25 0.25 0.25
    Li 0.75 0.75 0.75
    S 0 0 0
"""

StringHe = "He 0 0 0"

hydrogen_op = get_fermionic_op(
    atom_string = StringHe,
    spin        = 0,
    charge      = 0
)

hydrogen_ground_state = simulate_fermionic_op(hydrogen_op)

print("Hartree groundstate energy for Hydrogen:", hydrogen_ground_state)

Hartree groundstate energy for Hydrogen: -7.7256176739513425


## Finding higher energy eigenstates